# TNAD Complete Tutorial: From Basics to Advanced Usage

This comprehensive tutorial covers:
1. Understanding the core concepts (MPS, CFS, FGBS)
2. Basic usage and examples
3. Advanced hyperparameter tuning
4. Performance optimization
5. Custom use cases

**Estimated time**: 30-45 minutes

---

## Part 1: Setup and Imports

First, let's install and import all necessary libraries.

In [ ]:
# Install TNAD (if not already installed)
# !pip install -e ..

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict

from transformers import AutoModelForCausalLM, AutoTokenizer
from tnad import (
    FidelityGuidedBeamSearcher,
    MPSSequence,
    compute_cfs,
    compute_cfs_from_mps,
    analyze_coherence_spectrum,
)
from tnad.utils import get_device

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✓ All imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Part 2: Understanding Matrix Product States (MPS)

### What is an MPS?

A Matrix Product State is a tensor network representation that captures correlations in sequences:

```
Token sequence: ["The", "cat", "sat", "on", "mat"]
        ↓
MPS chain: A₁ — A₂ — A₃ — A₄ — A₅
           |    |    |    |    |
         [The][cat][sat][on][mat]
```

Each tensor Aᵢ has shape `[χ_left, d, χ_right]` where:
- **d**: Physical dimension (embedding dimension)
- **χ**: Bond dimension (controls entanglement capacity)

Let's build an MPS step by step:

In [ ]:
# Create an MPS with small dimensions for visualization
mps = MPSSequence(
    bond_dim=8,           # χ = 8 (small for demo)
    embedding_dim=32,     # d = 32 (smaller than real LLM embeddings)
    device='cpu',
    normalize_embeddings=True,
)

print(f"Created MPS: {mps}")
print(f"Bond dimension (χ): {mps.bond_dim}")
print(f"Embedding dimension (d): {mps.embedding_dim}")

### Adding Tokens to MPS

As we add tokens, the MPS grows and captures correlations:

In [ ]:
# Simulate adding 20 tokens (random embeddings for demonstration)
cfs_values = []
sequence_lengths = []

for i in range(20):
    # Simulate a token embedding (in practice, this comes from LLM)
    token_embedding = torch.randn(32)
    mps.add_token(token_embedding)
    
    # Compute Coherence Fidelity Score
    if mps.get_current_length() > 1:
        schmidt_values = mps.get_schmidt_values()
        cfs = compute_cfs(schmidt_values)
        cfs_values.append(cfs)
        sequence_lengths.append(mps.get_current_length())

print(f"Final MPS length: {mps.get_current_length()}")
print(f"Final CFS: {cfs_values[-1]:.3f}")

In [ ]:
# Visualize CFS evolution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: CFS over time
ax1.plot(sequence_lengths, cfs_values, marker='o', linewidth=2.5, markersize=8, color='#2E86AB')
ax1.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Min Coherence')
ax1.axhline(y=mps.bond_dim, color='green', linestyle='--', alpha=0.5, label=f'Max Coherence (χ={mps.bond_dim})')
ax1.set_xlabel('Sequence Length', fontsize=13, fontweight='bold')
ax1.set_ylabel('Coherence Fidelity Score (CFS)', fontsize=13, fontweight='bold')
ax1.set_title('CFS Evolution During Sequence Growth', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot 2: CFS distribution
ax2.hist(cfs_values, bins=15, color='#A23B72', alpha=0.7, edgecolor='black')
ax2.axvline(x=np.mean(cfs_values), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(cfs_values):.2f}')
ax2.set_xlabel('Coherence Fidelity Score', fontsize=13, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=13, fontweight='bold')
ax2.set_title('CFS Distribution', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"CFS Statistics:")
print(f"  Mean: {np.mean(cfs_values):.3f}")
print(f"  Std:  {np.std(cfs_values):.3f}")
print(f"  Range: [{np.min(cfs_values):.3f}, {np.max(cfs_values):.3f}]")

### Understanding Schmidt Values

Schmidt values measure entanglement between prefix and suffix of a sequence. Let's visualize them:

In [ ]:
# Get Schmidt values at the middle cut
schmidt_values = mps.get_schmidt_values()
analysis = analyze_coherence_spectrum(schmidt_values)

print("Schmidt Spectrum Analysis:")
print(f"  CFS: {analysis['cfs']:.3f}")
print(f"  Purity: {analysis['purity']:.4f}")
print(f"  Effective Rank: {analysis['effective_rank']:.2f}/{analysis['bond_dim']}")
print(f"  Entropy: {analysis['entropy']:.4f}")
print(f"  Max Schmidt: {analysis['max_schmidt']:.4f}")
print(f"  Uniformity: {analysis['uniformity']:.4f}")

# Visualize Schmidt spectrum
plt.figure(figsize=(10, 5))
plt.bar(range(len(schmidt_values)), schmidt_values, color='#F18F01', edgecolor='black', alpha=0.8)
plt.xlabel('Schmidt Index i', fontsize=13, fontweight='bold')
plt.ylabel('Schmidt Value λᵢ', fontsize=13, fontweight='bold')
plt.title('Schmidt Spectrum (Entanglement Structure)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Uniform spectrum → High entanglement → Coherent sequence")
print("- Peaked spectrum → Low entanglement → Decoherent sequence")

## Part 3: Fidelity-Guided Beam Search (FGBS)

Now let's use FGBS with a real language model. We'll start with a small model for quick demonstration.

In [ ]:
# Choose your model (adjust based on your hardware)
# Option 1: Small model (CPU-friendly)
model_name = "gpt2"  

# Option 2: Larger model (requires GPU and Hugging Face access)
# model_name = "meta-llama/Llama-3.1-8B-Instruct"

print(f"Loading model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

device = get_device()
if not torch.cuda.is_available():
    model = model.to(device)

print(f"✓ Model loaded on {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

### Initialize FGBS Searcher

Key hyperparameters:
- **beam_width (B)**: Number of parallel hypotheses (5-10 typical)
- **alpha (α)**: Fluency-coherence balance (0.5 = balanced)
- **bond_dim (χ)**: MPS entanglement capacity (16 typical)
- **top_k**: Candidate tokens per beam (50 typical)

In [ ]:
# Initialize FGBS with default hyperparameters
searcher = FidelityGuidedBeamSearcher(
    model=model,
    tokenizer=tokenizer,
    beam_width=5,           # B = 5 parallel beams
    alpha=0.5,              # α = 0.5 (balanced)
    bond_dim=16,            # χ = 16 (moderate logical bandwidth)
    top_k=50,               # Consider top-50 tokens
    temperature=1.0,        # No temperature scaling
    device=device,
)

print("✓ FGBS Searcher initialized")
print(f"  Configuration: B={searcher.beam_width}, α={searcher.alpha}, χ={searcher.bond_dim}")
print(f"  Embedding dimension: {searcher.embedding_dim}")

### Example 1: Math Reasoning

Test FGBS on a multi-step math problem.

In [ ]:
# Math problem requiring step-by-step reasoning
prompt_math = """Q: Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
A: Let's think step by step."""

print("Generating solution with FGBS...\n")
result_math = searcher.generate(
    prompt_math,
    max_length=150,
    min_length=30,
    return_details=True,
    show_progress=True,
)

print("\n" + "="*80)
print("GENERATED SOLUTION:")
print("="*80)
print(result_math['text'])
print("\n" + "="*80)
print("METRICS:")
print("="*80)
print(f"Log Probability (Fluency): {result_math['log_prob']:.2f}")
print(f"Log CFS (Coherence): {result_math['log_cfs']:.2f}")
print(f"CFS (exponential): {np.exp(result_math['log_cfs']):.2f}")
print(f"Composite Score: {result_math['composite_score']:.2f}")
print(f"Generated tokens: {len(result_math['token_ids'])}")

### Visualize Generation Dynamics

In [ ]:
# Extract trajectories
cfs_traj = result_math['cfs_trajectory']
score_traj = result_math['score_trajectory']

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: CFS trajectory
axes[0, 0].plot(cfs_traj, linewidth=2.5, color='#2E86AB', marker='o', markersize=4)
axes[0, 0].axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Min Coherence')
axes[0, 0].set_xlabel('Generation Step', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Coherence Fidelity Score', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Coherence Evolution (CFS)', fontsize=13, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Composite score trajectory
axes[0, 1].plot(score_traj, linewidth=2.5, color='#A23B72', marker='s', markersize=4)
axes[0, 1].set_xlabel('Generation Step', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Composite Score', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Composite Score (α·log P + (1-α)·log F)', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: CFS histogram
axes[1, 0].hist(cfs_traj, bins=20, color='#F18F01', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(x=np.mean(cfs_traj), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(cfs_traj):.2f}')
axes[1, 0].set_xlabel('Coherence Fidelity Score', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1, 0].set_title('CFS Distribution', fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Rolling statistics
window = 5
rolling_mean = np.convolve(cfs_traj, np.ones(window)/window, mode='valid')
axes[1, 1].plot(cfs_traj, alpha=0.3, color='gray', label='Raw CFS')
axes[1, 1].plot(range(window-1, len(cfs_traj)), rolling_mean, linewidth=2.5, color='#006BA6', label=f'Rolling Mean (w={window})')
axes[1, 1].set_xlabel('Generation Step', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('CFS', fontsize=12, fontweight='bold')
axes[1, 1].set_title('CFS with Rolling Average', fontsize=13, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nStatistics:")
print(f"  Average CFS: {np.mean(cfs_traj):.3f}")
print(f"  CFS std dev: {np.std(cfs_traj):.3f}")
print(f"  Min CFS: {np.min(cfs_traj):.3f}")
print(f"  Max CFS: {np.max(cfs_traj):.3f}")

## Part 4: Comparing FGBS with Baseline

Let's compare FGBS (α=0.5) with standard beam search (α=1.0).

In [ ]:
# Simple logical reasoning prompt
prompt_logic = "If A > B and B > C, then"

print("Running comparison: FGBS vs Standard Beam Search\n")
comparison = searcher.compare_with_baseline(
    prompt_logic,
    max_length=50,
)

print("\n" + "="*80)
print("FGBS OUTPUT (α=0.5 - Balanced):")
print("="*80)
print(comparison['fgbs']['text'])
print(f"\n  Final CFS: {np.exp(comparison['fgbs']['log_cfs']):.2f}")
print(f"  Log Probability: {comparison['fgbs']['log_prob']:.2f}")

print("\n" + "="*80)
print("BASELINE OUTPUT (α=1.0 - Pure LLM):")
print("="*80)
print(comparison['baseline']['text'])
print(f"\n  Final CFS: {np.exp(comparison['baseline']['log_cfs']):.2f}")
print(f"  Log Probability: {comparison['baseline']['log_prob']:.2f}")

print("\n" + "="*80)
print("IMPROVEMENT:")
print("="*80)
cfs_comp = comparison['cfs_comparison']
print(f"CFS Improvement: {cfs_comp['cfs_improvement']:.2f} ({cfs_comp['cfs_improvement']/cfs_comp['baseline_final_cfs']*100:.1f}%)")
print(f"FGBS achieves {cfs_comp['fgbs_final_cfs']/cfs_comp['baseline_final_cfs']:.2f}x higher coherence")

## Part 5: Hyperparameter Ablation Study

### Effect of Alpha (α): Fluency-Coherence Tradeoff

- α = 0: Pure coherence (may sacrifice fluency)
- α = 0.5: Balanced (recommended)
- α = 1: Pure LLM probability (standard beam search)

In [ ]:
# Test different alpha values
alphas = [0.0, 0.3, 0.5, 0.7, 1.0]
test_prompt = "The logical conclusion is"

alpha_results = []

print("Running alpha ablation study...\n")
for alpha in alphas:
    print(f"Testing α={alpha}...")
    
    # Create searcher with specific alpha
    test_searcher = FidelityGuidedBeamSearcher(
        model=model,
        tokenizer=tokenizer,
        beam_width=3,  # Smaller beam for speed
        alpha=alpha,
        bond_dim=16,
        device=device,
    )
    
    # Generate
    result = test_searcher.generate(
        test_prompt,
        max_length=40,
        show_progress=False,
    )
    
    alpha_results.append({
        'alpha': alpha,
        'text': result['text'],
        'log_prob': result['log_prob'],
        'cfs': np.exp(result['log_cfs']),
        'composite_score': result['composite_score'],
    })

print("\n✓ Ablation study complete")

In [ ]:
# Visualize alpha ablation results
alphas_list = [r['alpha'] for r in alpha_results]
cfs_list = [r['cfs'] for r in alpha_results]
logprob_list = [r['log_prob'] for r in alpha_results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Dual axis (CFS and Log Prob)
color1 = '#2E86AB'
ax1.plot(alphas_list, cfs_list, marker='o', linewidth=2.5, markersize=10, color=color1, label='CFS')
ax1.set_xlabel('Alpha (α)', fontsize=13, fontweight='bold')
ax1.set_ylabel('Coherence Fidelity Score', fontsize=13, fontweight='bold', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(True, alpha=0.3)
ax1.set_title('Effect of α on Fluency vs Coherence', fontsize=14, fontweight='bold')

ax1_twin = ax1.twinx()
color2 = '#A23B72'
ax1_twin.plot(alphas_list, logprob_list, marker='s', linewidth=2.5, markersize=10, color=color2, label='Log Prob')
ax1_twin.set_ylabel('Log Probability', fontsize=13, fontweight='bold', color=color2)
ax1_twin.tick_params(axis='y', labelcolor=color2)

# Add legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twin.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=11)

# Plot 2: Composite score
composite_list = [r['composite_score'] for r in alpha_results]
ax2.plot(alphas_list, composite_list, marker='D', linewidth=2.5, markersize=10, color='#F18F01')
ax2.set_xlabel('Alpha (α)', fontsize=13, fontweight='bold')
ax2.set_ylabel('Composite Score', fontsize=13, fontweight='bold')
ax2.set_title('Composite Score vs Alpha', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='α=0.5 (balanced)')
ax2.legend(fontsize=11)

plt.tight_layout()
plt.show()

# Print detailed results
print("\n" + "="*80)
print("ALPHA ABLATION RESULTS:")
print("="*80)
for r in alpha_results:
    print(f"\nα={r['alpha']:.1f}:")
    print(f"  CFS: {r['cfs']:.2f} | Log P: {r['log_prob']:.2f} | Composite: {r['composite_score']:.2f}")
    print(f"  Text: {r['text'][:100]}...")

print("\n" + "="*80)
print("KEY INSIGHTS:")
print("="*80)
print("- α=0.0: Maximizes coherence, may sacrifice fluency")
print("- α=0.5: Balanced approach (recommended for most tasks)")
print("- α=1.0: Standard beam search (fluency only)")
print("\nRecommendation: Use α=0.3-0.5 for reasoning tasks, α=0.6-0.7 for creative writing")

## Part 6: Custom Prompts and Use Cases

Now try your own prompts!

In [ ]:
# Custom prompt - edit this!
custom_prompt = """Q: If all cats are mammals, and all mammals are animals, what can we conclude about cats?
A: Let's reason step by step."""

# Generate
custom_result = searcher.generate(
    custom_prompt,
    max_length=120,
    return_details=True,
)

print("\n" + "="*80)
print("CUSTOM PROMPT RESULT:")
print("="*80)
print(custom_result['text'])
print("\n" + "="*80)
print("METRICS:")
print("="*80)
print(f"Final CFS: {np.exp(custom_result['log_cfs']):.2f}")
print(f"Generation length: {len(custom_result['token_ids'])} tokens")
print(f"Average CFS: {np.mean(custom_result['cfs_trajectory']):.2f}")
print(f"CFS stability (std): {np.std(custom_result['cfs_trajectory']):.3f}")

## Part 7: Performance Tips and Best Practices

### Hyperparameter Selection Guide

| Use Case | α | χ | B | Notes |
|----------|---|---|---|-------|
| **Math reasoning** | 0.3-0.4 | 16-32 | 5-8 | Prioritize coherence |
| **Logical reasoning** | 0.4-0.5 | 16 | 5 | Balanced |
| **Creative writing** | 0.6-0.8 | 8-16 | 3-5 | Prioritize fluency |
| **Fast generation** | 0.5 | 8 | 3 | Reduced complexity |
| **High quality** | 0.4 | 32 | 8 | Maximum coherence tracking |

### Memory Optimization

```python
# For long sequences or large models:
import gc
import torch

# Clear cache periodically
gc.collect()
torch.cuda.empty_cache()

# Use smaller bond dimension
searcher = FidelityGuidedBeamSearcher(
    ...,
    bond_dim=8,  # Instead of 16
    beam_width=3,  # Instead of 5
)
```

### Debugging Tips

```python
# Enable detailed logging
import logging
logging.basicConfig(level=logging.DEBUG)

# Inspect beam trajectories
result = searcher.generate(prompt, return_details=True)
all_beams = result['all_beams']
for i, beam in enumerate(all_beams):
    print(f"Beam {i}: score={beam.composite_score:.3f}, "
          f"cfs={np.exp(beam.log_cfs):.3f}")
```

## Summary and Next Steps

### What We Covered

1. ✅ **MPS fundamentals**: Tensor network representation of sequences
2. ✅ **CFS computation**: Coherence monitoring through Schmidt values
3. ✅ **FGBS algorithm**: Balancing fluency and coherence in generation
4. ✅ **Hyperparameter tuning**: Effect of α, χ, and B
5. ✅ **Performance optimization**: Best practices for efficient generation

### Further Resources

- 📄 **[Paper]**: Read the full technical paper
- 🔬 **[Experiments]**: Run GSM8K/StrategyQA benchmarks (`experiments/`)
- 📊 **[Performance Notebook]**: Detailed profiling (`notebooks/performance_benchmark.ipynb`)
- 🎮 **[Colab Demo]**: Try on Google Colab (`notebooks/tnad_colab.ipynb`)

### Citation

If you use TNAD in your research:

```bibtex
@article{tnad2024,
  title={TNAD: Tensor Network-Augmented Decoding for Coherent LLM Reasoning},
  author={Your Name},
  journal={arXiv preprint arXiv:2XXX.XXXXX},
  year={2024}
}
```

---

**Questions or issues?** Open an issue on [GitHub](https://github.com/your-username/quantum-searchllm/issues)

**Happy coherent generating! 🚀**